### From SQL Server to Pandas DataFrame

In [1]:
import pandas as pd
import pyodbc

** Lets check the pyodbc module documentaion **

In [2]:
pyodbc.__doc__

"A database module for accessing databases via ODBC.\n\nThis module conforms to the DB API 2.0 specification while providing\nnon-standard convenience features.  Only standard Python data types are used\nso additional DLLs are not required.\n\nStatic Variables:\n\nversion\n  The module version string.  Official builds will have a version in the format\n  `major.minor.revision`, such as 2.1.7.  Beta versions will have -beta appended,\n  such as 2.1.8-beta03.  (This would be a build before the official 2.1.8 release.)\n  Some special test builds will have a test name (the git branch name) prepended,\n  such as fixissue90-2.1.8-beta03.\n\napilevel\n  The string constant '2.0' indicating this module supports DB API level 2.0.\n\nlowercase\n  A Boolean that controls whether column names in result rows are lowercased.\n  This can be changed any time and affects queries executed after the change.\n  The default is False.  This can be useful when database columns have\n  inconsistent capitaliz

**Lets check what's inside the pyodbc module**

In [3]:
dir(pyodbc)

['BINARY',
 'Binary',
 'BinaryNull',
 'Connection',
 'Cursor',
 'DATETIME',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NUMBER',
 'NotSupportedError',
 'OperationalError',
 'ProgrammingError',
 'ROWID',
 'Row',
 'SQLWCHAR_SIZE',
 'SQL_ACCESSIBLE_PROCEDURES',
 'SQL_ACCESSIBLE_TABLES',
 'SQL_ACTIVE_ENVIRONMENTS',
 'SQL_AGGREGATE_FUNCTIONS',
 'SQL_ALTER_DOMAIN',
 'SQL_ALTER_TABLE',
 'SQL_ASYNC_MODE',
 'SQL_ATTR_TXN_ISOLATION',
 'SQL_BATCH_ROW_COUNT',
 'SQL_BATCH_SUPPORT',
 'SQL_BIGINT',
 'SQL_BINARY',
 'SQL_BIT',
 'SQL_BOOKMARK_PERSISTENCE',
 'SQL_CATALOG_LOCATION',
 'SQL_CATALOG_NAME',
 'SQL_CATALOG_NAME_SEPARATOR',
 'SQL_CATALOG_TERM',
 'SQL_CATALOG_USAGE',
 'SQL_CHAR',
 'SQL_COLLATION_SEQ',
 'SQL_COLUMN_ALIAS',
 'SQL_CONCAT_NULL_BEHAVIOR',
 'SQL_CONVERT_FUNCTIONS',
 'SQL_CONVERT_VARCHAR',
 'SQL_CORRELATION_NAME',
 'SQL_CREATE_ASSERTION',
 'SQL_CREATE_CHARACTER_SET',
 'SQL_CREATE_COLLATION',
 'SQL_CREATE_

**First lets check our SQL Server driver version**

In [4]:
pyodbc.drivers()

['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'SQL Server Native Client 11.0',
 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)',
 'ODBC Driver 13 for SQL Server',
 'SQL Server Native Client RDA 11.0',
 'SQL Server Native Client 10.0',
 'PostgreSQL ANSI(x64)',
 'PostgreSQL Unicode(x64)',
 'Amazon Redshift (x64)',
 'MySQL ODBC 8.0 ANSI Driver',
 'MySQL ODBC 8.0 Unicode Driver']

**we'll create a connection to SQL Server**

In [5]:
sql_conn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER=DESKTOP-O6725QL;DATABASE=Chinook;Trusted_Connection=yes')

**We'll query the Chinook database:**

**Your task is to create a pandas DataFrame that displays Customers’ Last Name and First Name, and each customer’s purchased Track names and Album Titles. The information should be sorted by Customer LastName then Customer FirstName. The first five rows of your results should look like this:**

In [6]:
qry = "SELECT * FROM Chinook.dbo.VW_Chinook_qry1 Order By LastName, FirstName"

In [7]:
df = pd.read_sql(qry, sql_conn)

In [8]:
df.head()

,LastName,FirstName,Name,Title
0,Almeida,Roberto,Vai Passar,Minha Historia
1,Almeida,Roberto,Morena De Angola,Minha Historia
2,Almeida,Roberto,Meu Caro Amigo,Minha Historia
3,Almeida,Roberto,Mateus Enter,Afrociberdelia
4,Almeida,Roberto,Samba Do Lado,Afrociberdelia


In [9]:
df.tail()

,LastName,FirstName,Name,Title
2235,Zimmermann,Fynn,Nothing to Hide,"Heroes, Season 1"
2236,Zimmermann,Fynn,Eu E Ela,Volume Dois
2237,Zimmermann,Fynn,"Exodus, Pt. 1","Battlestar Galactica, Season 3"
2238,Zimmermann,Fynn,Amanhã Não Se Sabe,Volume Dois
2239,Zimmermann,Fynn,Sonifera Ilha,Volume Dois


**The SQL query for the view**

```sql
with songs as (
	SELECT [TrackId]
			,a.[Name] as 'Track_Name'
			,b.Title as 'Album_Title'
			,c.ArtistId
			,c.Name as 'Artist'
	FROM [dbo].[Track] a
	left outer join Album b
		on a.AlbumId = b.AlbumId
	left outer join	 Artist c
		on b.ArtistId = c.ArtistId
)
select LastName
		,FirstName
		,s.Track_Name as 'Name'
		,s.Album_Title as 'Title'
from Customer c 
left outer join Invoice i
	on c.CustomerId = i.CustomerId
left outer join InvoiceLine il
	on i.InvoiceId = il.InvoiceId
left outer join songs s
	on il.TrackId = s.TrackId
```